In [1]:
# Jupyter notebook test file for actual python files
# Andrew J Miller
# Jun 9 2017

# Follow allong for Yoan Blanc's Medium article

In [2]:
"""
    This is for testing the Slack API authentication for out bot. (api.py)
""";

In [3]:
# Import libraries
import asyncio
import aiohttp

In [4]:
# Import our config file.
# This is just to hide the Token file.
from config import DEBUG, TOKEN

In [5]:
# You should store your actual files here after uncommenting this.
# DEBUG = True
# TOKEN = "xoxo-.."

In [6]:
async def api_call(method, data = None, token = TOKEN):
    # This calls the slack API.
    
    # Acts on aiohttp.Client as session
    with aiohttp.ClientSession() as session:
        
        # Creates our form.
        form = aiohttp.FormData(data or {})
        
        # Adds our token to the form.
        form.add_field('token', token)
        
        # This is a coroutine, it is a function that stops and resumes based on the awaoy response below.
        # 
        async with session.post('https://slack.com/api/{0}'.format(method),
                                data = form) as response:
            assert 200 == response.status, ('{0} with {1} failed.'.format(method, data))
            
            return await response.json()

In [7]:
# This is the main run block.
# if you get an ok it means your token key is working.

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.set_debug(DEBUG)
    response = loop.run_until_complete(api_call('auth.test'))
    loop.close()

    assert response['ok']
    print(response)

Executing <Task pending coro=<api_call() running at <ipython-input-6-f19a7f6f2c57>:16> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at C:\Users\andym\Anaconda3\envs\aind\lib\asyncio\futures.py:407, <TaskWakeupMethWrapper object at 0x0000020A25A9AD98>()] created at C:\Users\andym\Anaconda3\envs\aind\lib\asyncio\base_events.py:275> cb=[_run_until_complete_cb() at C:\Users\andym\Anaconda3\envs\aind\lib\asyncio\base_events.py:176] created at C:\Users\andym\Anaconda3\envs\aind\lib\asyncio\base_events.py:446> took 0.234 seconds


{'ok': True, 'url': 'https://testing1313.slack.com/', 'team': 'testing', 'user': 'andros', 'team_id': 'T65U8K7K5', 'user_id': 'U65AW2G72'}


In [17]:
# But what about for connecting the actual bot?

In [18]:
"""
    Bot that prints to the console (andros.py)
""";

In [19]:
# Import libraries
import asyncio
import json
import sys
import aiohttp

In [20]:
# from api import api_call
from config import DEBUG, TOKEN

In [21]:
async def andros(token = TOKEN):
    """
        Creates a bot that connects with Slack Real Time Messaging API
    """
    rtm = await api_call("rtm.start")
    assert rtm['ok'], "Connection error with RTM API"
    
    async with aiohttp.ClientSession() as session:
        async with session.ws_connect(rtm["url"]) as ws:
            async for msg in ws:
                assert msg.tp == aiohttp.MsgType.text
                print(json.loads(msg.data))

In [22]:
# Run Block
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.set_debug(DEBUG)
    loop.run_until_complete(andros())
    loop.close()

RuntimeError: Event loop is closed

C:\Users\andym\Anaconda3\envs\aind\lib\site-packages\simplejson\encoder.py:371: RuntimeWarning: coroutine 'andros' was never awaited
  self.iterable_as_array, Decimal=decimal.Decimal)


In [23]:
# Let's make another iteration of the above that actually does something other than
# printing JSON information to the console.

In [24]:
# Import libraries
import asyncio
import json
import sys
import aiohttp
# from api import api_call
from config import DEBUG, TOKEN

In [ ]:
# This is pretty obvious to see.
# This takes in the messaege as long as it's a message and prints it out.
# If there's an error it lets us know.

async def consumer(message):
    """
        Display recieved message
    """
    if message.get('type') == 'message':
        user = await api_call('users.info', {'user': message.get('user')})
        
        print("{0}: {1}".format(user["user"]["name"], message["text"]))
        
        else:
            print(message, file=sys.stderr)

In [ ]:
# This connects but notice the added aysyncio funciton.
async def andros(token = TOKEN):
    """
        Creates a bot that connects with Slack Real Time Messaging API
    """
    rtm = await api_call("rtm.start")
    assert rtm['ok'], "Connection error with RTM API"
    
    async with aiohttp.ClientSession() as session:
        async with session.ws_connect(rtm["url"]) as ws:
            async for msg in ws:
                assert msg.tp == aiohttp.MsgType.text
                print(json.loads(msg.data))
                asyncio.ensure_future(consumer(message))